<a href="https://colab.research.google.com/github/assafrbi/mini-project-Digital-Humanities/blob/main/dh_mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Python Package imports
import requests
from bs4 import BeautifulSoup
from dateutil.parser import parse
import concurrent.futures
import pandas as pd

In [2]:
# Maximum number of threads that will be spawned
MAX_THREADS = 50

### Load Data

In [3]:
movie_title_arr = []
movie_year_arr = []
movie_genre_arr = []
movie_synopsis_arr =[]
image_url_arr  = []
image_id_arr = []

In [4]:
def getMovieTitle(header):
    try:
        return header[0].find("a").getText()
    except:
        return 'NA'

def getReleaseYear(header):
    try:
        return header[0].find("span",  {"class": "lister-item-year text-muted unbold"}).getText()
    except:
        return 'NA'

def getGenre(muted_text):
    try:
        return muted_text.find("span",  {"class":  "genre"}).getText()
    except:
        return 'NA'

def getsynopsys(movie):
    try:
        return movie.find_all("p", {"class":  "text-muted"})[1].getText()
    except:
        return 'NA'

def getImage(image):
    try:
        return image.get('loadlate')
    except:
        return 'NA'

def getImageId(image):
    try:
        return image.get('data-tconst')
    except:
        return 'NA'

In [5]:
def main(imdb_url):
    response = requests.get(imdb_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Movie Name
    movies_list  = soup.find_all("div", {"class": "lister-item mode-advanced"})
    
    for movie in movies_list:
        header = movie.find_all("h3", {"class":  "lister-item-header"})
        muted_text = movie.find_all("p", {"class":  "text-muted"})[0]
        imageDiv =  movie.find("div", {"class": "lister-item-image float-left"})
        image = imageDiv.find("img", "loadlate")
        
        #  Movie Title
        movie_title =  getMovieTitle(header)
        movie_title_arr.append(movie_title)
        
        #  Movie release year
        year = getReleaseYear(header)
        movie_year_arr.append(year)
        
        #  Genre  of movie
        genre = getGenre(muted_text)
        movie_genre_arr.append(genre)
        
        # Movie Synopsys
        synopsis = getsynopsys(movie)
        movie_synopsis_arr.append(synopsis)
        
        #  Image attributes
        img_url = getImage(image)
        image_url_arr.append(img_url)
        
        image_id = image.get('data-tconst')
        image_id_arr.append(image_id)

In [6]:
# An array to store all the URL that are being queried
imageArr = []

# Maximum number of pages one wants to iterate over
MAX_PAGE =51

# Loop to generate all the URLS.
for i in range(1, 1500, 50):
    imdb_url = f'https://www.imdb.com/search/title/?title_type=feature&countries=il&start={i}&ref_=adv_nxt'
    imageArr.append(imdb_url)

In [7]:
def download_stories(story_urls):
    threads = min(MAX_THREADS, len(story_urls))
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
        executor.map(main, story_urls)

In [8]:
# Call the download function with the array of URLS called imageArr
download_stories(imageArr)

# Attach all the data to the pandas dataframe. You can optionally write it to a CSV file as well
movieDf = pd.DataFrame({
    "Title": movie_title_arr,
    "Release_Year": movie_year_arr,
    "Genre": movie_genre_arr,
    "Synopsis": movie_synopsis_arr,
    "image_url": image_url_arr,
    "image_id": image_id_arr,
})

print('--------- Download Complete CSV Formed --------')

# movie.to_csv('file.csv', index=False) : If you want to store the file.
movieDf

--------- Download Complete CSV Formed --------


,Title,Release_Year,Genre,Synopsis,image_url,image_id
0,Untitled Horatio Spafford Project,,"\nBiography, Drama, War",\nA famous Chicago lawyer loses family and for...,https://m.media-amazon.com/images/M/MV5BZGFkZj...,tt13572382
1,The Untitled Matan Shiram Project,,\nDrama,\nImpressions from moments of observation. For...,https://m.media-amazon.com/images/M/MV5BYjgwND...,tt11101452
2,Landsman,(2009),\nDrama,\nSeventy-eight-year-old Ze'ev Landsman gives ...,https://m.media-amazon.com/images/M/MV5BZGIxMW...,tt1438464
3,Visa,(I) (2013),"\nComedy, Drama",\nA sham marriage for the sake of a UK residen...,https://m.media-amazon.com/images/M/MV5BMjNiNj...,tt3002628
4,Funeral at Noon,(2013),\nDrama,\nA young housewife yearns for human contact i...,https://m.media-amazon.com/images/M/MV5BMTY4Nz...,tt3114224
...,...,...,...,...,...,...
1466,Trailer WoW Movie I - The Battle of Hillsbrad,(2022),\nComedy,\n2 Israeli Orthodox Jews advertise themselves...,https://m.media-amazon.com/images/S/sash/NapCx...,tt4365542
1467,Marco and Polo,,"\nMusic, Romance",\nA look at the chaos and confusion of love fe...,https://m.media-amazon.com/images/M/MV5BNTZlY2...,tt12398004
1468,Love Chapter 2,(2017),\nDrama,"\nA highly successful lawyer realises, after a...",https://m.media-amazon.com/images/M/MV5BZTM3Mj...,tt0089931
1469,"Again, Forever",(1985),\nDrama,\nMichale is a thirty year old woman. She work...,https://m.media-amazon.com/images/M/MV5BMjE1MD...,tt0362427


### Text Preprocessing


### Topic Modeling - Latent Dirichlet Allocation